In [ ]:
# syft absolute
import syft as sy
from syft.service.code.user_code import UserCode
from syft.service.job.job_stash import Job
from syft.service.request.request import Request

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
request = high_client.requests[0]

In [ ]:
request

In [ ]:
request.code

In [ ]:
def execute_request(client_high, request) -> dict:
    if not isinstance(request, Request):
        return "This is not a request"

    code = request.code
    if not isinstance(code, UserCode):
        return "No usercode found"

    func_name = request.code.service_func_name
    api_func = getattr(client_high.code, func_name, None)
    if api_func is None:
        return "Code name was not found on the client."

    job = api_func(blocking=False)
    return job

In [ ]:
job = execute_request(high_client, request)

In [ ]:
job

In [ ]:
isinstance(job, Job)

In [ ]:
result = job.wait()
result

In [ ]:
assert len(result) == 10000

In [ ]:
job_info = job.info(result=True)
response = request.deposit_result(job_info, approve=True)
response

In [ ]:
isinstance(response, Job)

In [ ]:
def list_pending(do_client):
    results = []
    for request in do_client.requests:
        if str(request.status) == "RequestStatus.PENDING":
            results.append(request)
            print(request.id)
            print(request.code.constants["query"].val)
            print()
    return results

In [ ]:
def approve_and_deposit(do_client, request_id_or_idx):
    if isinstance(request_id_or_idx, int):
        pending = list_pending(do_client)
        request = pending[request_id_or_idx]
    elif isinstance(request_id_or_idx, str):
        request = do_client.requests.get_by_uid(uid=sy.UID(request_id_or_idx))
    else:
        raise Exception(f"Wrong type for request_id_or_idx: {type(request_id_or_idx)}")

    if not isinstance(request, Request):
        return "This is not a request"

    code = request.code
    if not isinstance(code, UserCode):
        return "No usercode found on request"

    func_name = request.code.service_func_name
    api_func = getattr(do_client.code, func_name, None)
    if api_func is None:
        return "Code name not found in the client's code"

    job = api_func(blocking=False)
    job.wait()

    job_info = job.info(result=True)
    return request.deposit_result(job_info, approve=True)

In [ ]:
pending = list_pending(high_client)
pending

In [ ]:
result = approve_and_deposit(high_client, 0)

In [ ]:
assert isinstance(result, Job)

In [ ]:
server.land()